In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

In [ ]:
url = 'https://raw.githubusercontent.com/digipodium/Datasets/main/classfication/diabetes.csv'
df = pd.read_csv(url)
df.head()

In [ ]:
df.isnull().sum()     # no missing values

pip install imblearn to handle class - imbalance

In [ ]:
df.Outcome.value_counts().plot(kind='pie', autopct='%.1f%%', 
    labels=['No Diabetes', 'Diabetes'],
    colors=['skyblue', 'lightgreen'], explode=[0, 0.05])

Importing Libraries

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, classification_report,ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [ ]:
X = df.drop('Outcome', axis=1)
y = df['Outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print("training data =>",X_train.shape, X_test.shape)

In [ ]:
model = Pipeline([
    ('scaler', StandardScaler()),
    ('dt', DecisionTreeClassifier())
])
model

In [ ]:
list(range(5, 100, 10))      # just to see the range

In [ ]:
# `dt` is the name of the decision tree classifier in the pipeline
params = {
    'dt__criterion': ['gini', 'entropy', 'log_loss'],
    'dt__max_depth': list(range(5, 100, 10))
}

grid = GridSearchCV(model, param_grid=params, cv=5, verbose=1)
grid

Fitting the model

In [ ]:
grid.fit(X_train, y_train)

Best Parameters

In [ ]:
grid.best_params_

All the metrics as a dataframe

In [ ]:
result_df = pd.DataFrame( grid.cv_results_ )
result_df.sort_values('rank_test_score')

Extract the best model

In [ ]:
best_model = grid.best_estimator_
best_model

Check the confusion matrix

In [ ]:
y_pred = best_model.predict(X_test)
cf = confusion_matrix(y_test, y_pred)
cf

In [ ]:
fig, ax = plt.subplots(figsize=(4, 4))
ConfusionMatrixDisplay(cf, display_labels=['No Diabetes', 'Diabetes']).plot(ax=ax, colorbar=False)

In [ ]:
print(classification_report(y_test, y_pred))

Plotting the Grid Search Results

In [ ]:
result_df[['mean_test_score']].plot(kind='bar', logy=True, figsize=(10, 5))

In [ ]:
import joblib

joblib.dump(best_model, 'diabetes_model.pkl')

Understand Undersampling

In [ ]:
!pip install imblearn

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

In [ ]:
fig, (ax1, ax2) = plt.subplots(figsize=(8, 4), nrows=1, ncols=2)
rus = RandomUnderSampler()
print("before undersample")
print(y_train.value_counts())
y_train.value_counts().plot(kind='pie', 
    autopct='%.1f%%', 
    labels=['No Diabetes', 'Diabetes'],
    colors=['skyblue', 'lightgreen'], 
    explode=[0, 0.05],
    ax=ax1)

X_train, y_train = rus.fit_resample(X_train, y_train)
print("after undersample")
print(y_train.value_counts())
y_train.value_counts().plot(kind='pie', 
    autopct='%.1f%%', 
    labels=['No Diabetes', 'Diabetes'],
    colors=['skyblue', 'lightgreen'], 
    explode=[0, 0.05],
    ax=ax2)